In [56]:
import requests
import re
from bs4 import BeautifulSoup

In [1]:
from lempel_ziv_complexity import lempel_ziv_complexity

Info: numba.jit seems to be available.


In [28]:
lempel_ziv_complexity(s)

59

In [27]:
s = '''baby I dont need dollar bills to have fun tonight
I love cheap thrills
baby I dont need dollar bills to have fun tonight
I love cheap thrills
I dont need no money
as long as I can feel the beat
I dont need no money
as long as I keep dancing'''

In [21]:
(240-59)/240

0.7541666666666667

In [40]:
join = "".join(s.split())

189

In [31]:
def lzw_encode(data):
    code, code_bits = {bytes([i]): i for i in range(256)}, 8
    buffer, buffer_bits = 0, 0
    index, aux = 0, []
    while index < len(data):
        # find word
        for j in range(index + 1, len(data) + 1):
            word = data[index:j]
            # store word
            if word not in code:
                code[word] = len(code)
                word = word[:-1]
                break
        # write buffer
        buffer <<= code_bits
        buffer |= code[word]
        buffer_bits += code_bits
        # code length
        if len(code) > 2 ** code_bits:
            code_bits += 1
        # shift
        index += len(word)
        # buffer alignment
        if index >= len(data) and buffer_bits % 8:
            r = 8 - (buffer_bits % 8)
            buffer <<= r
            buffer_bits += r
        # emit output
        if not buffer_bits % 8:
            aux += int.to_bytes(buffer, buffer_bits >> 3, 'big')
            buffer, buffer_bits = 0, 0
    return bytes(aux)

In [32]:
def compress(data):
    encoded = lzw_encode(data.encode('ASCII'))
    decoded = lzw_decode(encoded).decode('ASCII')
    assert data == decoded
    
    print('compression', len(data), '->', len(encoded), 'bytes')

In [33]:
def lzw_decode(data):
    code, code_bits = {i: bytes([i]) for i in range(256)}, 8
    buffer, buffer_bits = 0, 0
    index, aux = 0, []
    prefix = b''

    while index < len(data) or buffer_bits >= code_bits:
        # read buffer
        while index < len(data) and buffer_bits < code_bits:
            buffer <<= 8
            buffer |= data[index]
            buffer_bits += 8
            index += 1

        # find word
        buffer_bits -= code_bits
        key = buffer >> buffer_bits
        buffer &= (1 << buffer_bits) - 1
        word = code.get(key, prefix + prefix[:1])

        # store word
        if prefix:
            code[len(code)] = prefix + word[:1]
        prefix = word

        # code length
        if len(code) >= 2 ** code_bits:
            code_bits += 1

        # emit output
        aux += word
        
    return bytes(aux)

In [41]:
compress(join)

compression 189 -> 143 bytes


In [43]:
143/189

0.7566137566137566

In [37]:
(240-169)/240

0.29583333333333334

In [38]:
x = '''tonight I need dollar bills
I don't keep fun
cheap thrills long to feel money
the bills don't need the dancing baby
fun dollar dancing thrills the baby I need
don't have fun
no no don't have dancing fun tonight
beat the can as I don't feel thrills
love the dancing money'''

In [39]:
compress(x)

compression 270 -> 183 bytes
